In [12]:
using Distributions
using Interpolations
using QuantEcon
using Optim

In [2]:
type LogLinearGrowthModel
    alpha::Float64
    beta::Float64
    f::Function
    u::Function
    mu::Float64
    sigma::Float64
    grid_max::Float64
    grid_size::Int64
    grid::LinSpace{Float64}
    shocks::Array{Float64}
    
    function LogLinearGrowthModel(alpha=0.65, beta=0.95, f=k -> k^alpha, u=log, mu=1, 
            sigma=0.1, grid_max=8.0, grid_size=150)
        
        new(alpha, beta, f, u, mu, sigma, grid_max, grid_size, 
            linspace(1e-6, grid_max, grid_size), rand(LogNormal(mu, sigma),250))
    end
    
end

In [27]:
function bellman_operator!(gm::LogLinearGrowthModel, w::Vector, out::Vector; ret_policy::Bool=false)
    
    Aw = scale(interpolate(w, BSpline(Linear()), OnGrid()), gm.grid)
    
    for (i, y) in enumerate(gm.grid)
        objective(c) = - gm.u(c) - gm.beta * mean([Aw[x] for x in gm.f(y-c) * gm.shocks])
        res = optimize(objective, 1e-7, y)
        c_star = res.minimum

        if ret_policy
            out[i] = c_star
        else
            out[i] = - objective(c_star)
        end
    end

    return out
end

function bellman_operator(gm::LogLinearGrowthModel, w::Vector;
                          ret_policy::Bool=false)
    out = similar(w)
    bellman_operator!(gm, w, out, ret_policy=ret_policy)
end

function get_greedy!(gm::LogLinearGrowthModel, w::Vector, out::Vector)
    bellman_operator!(gm, w, out, ret_policy=true)
end

get_greedy(gm::LogLinearGrowthModel, w::Vector) = bellman_operator(gm, w, ret_policy=true)

get_greedy (generic function with 1 method)

In [28]:
function compute_value_function(gm::LogLinearGrowthModel)
    Tw = zeros(length(gm.grid))
    initial_w = 5 * log(gm.grid) - 25
    bellman(w) = bellman_operator(gm, w)
    
    v_star = compute_fixed_point(bellman,
            initial_w, 
            err_tol=1e-4,
            max_iter=100,
            verbose=true,
            print_skip=5)
    
    return v_star
end

compute_value_function (generic function with 1 method)

In [29]:
gm = LogLinearGrowthModel();

In [30]:
v_star = compute_value_function(gm);

Compute iterate 5 with error 7.385666550300343
Compute iterate 10 with error 5.6644201229782425
Compute iterate 15 with error 4.331026788692782
Compute iterate 20 with error 3.304268067547355
Compute iterate 25 with error 2.517020883920111
Compute iterate 30 with error 1.9152039291335257
Compute iterate 35 with error 1.4560922107549175
Compute iterate 40 with error 1.1063618384838207
Compute iterate 45 with error 0.8402376447722588
Compute iterate 50 with error 0.6378931976491913
Compute iterate 55 with error 0.48413511866453973
Compute iterate 60 with error 0.36735096256239785
Compute iterate 65 with error 0.2786818774614517
Compute iterate 70 with error 0.2113789167744926
Compute iterate 75 with error 0.1603057560011507
Compute iterate 80 with error 0.1215563671213431
Compute iterate 85 with error 0.09216207402232612
Compute iterate 90 with error 0.06986764193499084
Compute iterate 95 with error 0.05296043904337466
Compute iterate 100 with error 0.040140264326282704
